# grocery_sales_forecasting_Main

In [1]:
# Import libraries necessary for this project
import os.path
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import ShuffleSplit
from IPython.display import display
import matplotlib.pyplot as plt

# Import supplementary visualizations code visuals.py
#import visuals as vs

# Pretty display for notebooks
%matplotlib inline

Load one year's train data;   
Add oil price of corresponding date;  
Add transaction of coresponding date and store;

In [2]:
import datetime
years = [2013, 2014, 2015, 2016]

#Load data
# train_2013 = pickle.load(open( 'train_2013.pkl', "rb" ))
# train_2014 = pickle.load(open( 'train_2014.pkl', "rb" ))
# train_2015 = pickle.load(open( 'train_2015.pkl', "rb" ))
# train_2016 = pickle.load(open( 'train_2016.pkl', "rb" ))
# train_2017 = pickle.load(open( 'train_2017.pkl', "rb" ))
# train_data = pd.concat([train_2013, train_2014, train_2015, train_2016, train_2017])

test_data = pd.read_csv("input/test.csv")
print("Favorita grocery sales forecasting testing data has {} samples with {} features each.".format(*test_data.shape))
test_data['date'] = pd.to_datetime(test_data['date'])

for year in years:
    pickle_file = 'train_' + str(year) + '_8.pkl'
    train_data = pickle.load(open( pickle_file, "rb" ))
    for item in range(test_data.shape[0]):
        date_test = test_data.at[item, 'date']
        test_store_nbr = test_data.at[item,'store_nbr']
        test_item_nbr = test_data.at[item,'item_nbr']

        date_hist = date_test.replace(year = year)
        mask = (train_data['date']==date_hist)&(train_data['store_nbr'] == test_store_nbr)&(train_data['item_nbr'] == test_item_nbr)
        result = train_data.loc[mask, 'unit_sales']
        if result.shape[0]>0:
            test_data.at[item, str(year)] = result.iloc[0]
    print(year, 'is completed!')
            
# test_data.loc[:,['id','unit_sales']].to_csv('submission.csv', sep='\t')
test_data.to_csv('test_data.csv', sep='\t')


Favorita grocery sales forecasting testing data has 3370464 samples with 5 features each.


KeyboardInterrupt: 

In [ ]:
#Load one year's train data;  
train_pkl = 'train_2017.pkl'
years = ['2013', '2014', '2015', '2016', '2017']
train_data = pickle.load(open( train_pkl, "rb" ))

#Add oil price of corresponding date; 
oil = pd.read_csv("input/oil.csv")
oil['date'] = pd.to_datetime(oil['date'], format='%Y-%m-%d')
train_data= pd.merge(train_data,oil, right_on='date',left_on='date',how='left')

#Add transaction of coresponding date and store;
transactions = pd.read_csv("input/transactions.csv")
transactions['date'] = pd.to_datetime(transactions['date'], format='%Y-%m-%d')
train_data= pd.merge(train_data,transactions, left_on=['date', 'store_nbr'], right_on=['date', 'store_nbr'], how='left')
display(train_data.tail(5))
display(train_data.describe())

In [ ]:
# Separate training feature and training target
target_raw = train_data['unit_sales']
features_raw = train_data.drop(['id', 'unit_sales'], axis=1)

#Drop items with missing data
features_raw.replace(["NaN", 'NaT'], np.nan, inplace = True)
features_raw.dropna(inplace=True)
# features_clean_NaN = features_raw[~features_raw.isin(['NaN', 'NaT']).any(axis=1)] #computationally expensive

#Categorical feature
features_raw['onpromotion'] = features_raw['onpromotion'].apply(lambda x: 1 if x else 0)

#Numerical feature
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() # default=(0, 1)
numerical = ['dcoilwtico', 'transactions']
features_minmax_transform = pd.DataFrame(data = features_raw)
features_minmax_transform[numerical] = scaler.fit_transform(features_minmax_transform[numerical])
display(features_minmax_transform.tail(5))
display(features_minmax_transform.describe())

Naive Model: Search the history record, and average it